<a href="https://colab.research.google.com/github/ruchi0451/NLP_AD/blob/main/Covid_Fake_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [29]:
df = pd.read_csv('/content/covid_fake.csv')

In [30]:
df.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [31]:
df.shape

(1164, 4)

In [33]:
df['label'].value_counts()

,count
label,
TRUE,584
Fake,345
fake,230


In [34]:
df.loc[5:15]

,title,text,source,label
5,CORONA UNMASKED: Chinese Intelligence Officer ...,NaN,NaN,NaN
6,NaN,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,Fake
7,NaN,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,Fake
8,NaN,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,Fake
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE
10,NaN,The new Coronavirus may not show signs of infe...,Taiwan Experts,Fake
11,NaN,A vaccine meant for cattle can be used to figh...,facebook,Fake
12,NaN,Using a hair dryer to breathe in hot air can c...,Youtube,Fake
13,NaN,Corona virus before it reaches the lungs it re...,twitter,Fake
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE


In [35]:
df.isna().sum()

,0
title,82
text,10
source,20
label,5


In [36]:
df.loc[df['label'] == 'Fake', ['label']] = 'FAKE'
df.loc[df['label'] == 'Fake', ['label']] = 'FAKE'
df.loc[df['label'] == 'facebook', ['source']] = 'Facebook'
df.text.fillna(df.title, inplace=True)
df.loc[5]['label'] = 'FAKE'
df.loc[15]['label'] = 'TRUE'
df.loc[43]['label'] = 'FAKE'
df.loc[131]['label'] = 'TRUE'
df.loc[242]['label'] = 'FAKE'
df = df.sample(frac=1).reset_index(drop=True)
df.title.fillna('missing', inplace=True)
df.source.fillna('missing', inplace=True)
df['title_text'] = df['title'] + '' + df['text']


In [37]:
df.isna().sum()

,0
title,0
text,0
source,0
label,0
title_text,0


In [38]:
df['label'].value_counts()

,count
label,
TRUE,586
FAKE,348
fake,230


In [39]:
df.head()

,title,text,source,label,title_text
0,Covid-19: Has Bill Gates Predicted the Current...,Exceptional situations like the current Covid ...,https://de.sputniknews.com/,fake,Covid-19: Has Bill Gates Predicted the Current...
1,"False claim: ""the Coronavirus"" is designed and...",A Facebook group shared an image on Facebook t...,https://www.reuters.com/,TRUE,"False claim: ""the Coronavirus"" is designed and..."
2,missing,Essential Oils to Protect Against Coronavirus ...,https://www.quinessence.com/,fake,missingEssential Oils to Protect Against Coron...
3,Will holding your breath for 10 seconds reveal...,University of Maryland Chief Quality Officer a...,https://www.usatoday.com/,TRUE,Will holding your breath for 10 seconds reveal...
4,Coronavirus: Covert euthanasia with the use of...,"In the interview, de Monredon said that a rec...",http://www.contre-info.com,fake,Coronavirus: Covert euthanasia with the use of...


In [40]:
df.shape

(1164, 5)

In [41]:
df['title_text'][3]

'Will holding your breath for 10 seconds reveal if you have coronavirus?University of Maryland Chief Quality Officer and Chief of Infectious Diseases Dr. Faheem Younus tweeted on March 16: “Wrong: Most young patients with coronavirus will be able to hold their breaths for much longer than 10 seconds. And many elderly without the virus won’t be able to do it.”On March 17, Dr. Thomas Nash, a New York Presbyterian Hospital internist, pulmonologist and infectious disease specialist, told Reuters the breath test was “just made up.”\nDifferent posts mistakenly source the claim to an unnamed Stanford Hospital Board member, a Japanese doctor or Taiwanese experts. On March 13, Stanford University tweeted, “Misinformation about COVID-19 symptoms and treatment falsely attributed to Stanford is circulating on social media and in email forwards. It is not from Stanford.”Stanford Health Care spokeswoman Lisa Kim told CNN on March 17 that the "dangerous" claim was not from Stanford Medicine and "cont

In [42]:
def preprocessor(text):
  text = re.sub('<[^>]*>', '', text)
  text =re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'[\n]', '', text)
  text = text.lower()
  return text
df['title_text'] = df['title_text'].apply(preprocessor)
df['title_text'][3]

'will holding your breath for 10 seconds reveal if you have coronavirusuniversity of maryland chief quality officer and chief of infectious diseases dr faheem younus tweeted on march 16 wrong most young patients with coronavirus will be able to hold their breaths for much longer than 10 seconds and many elderly without the virus wont be able to do iton march 17 dr thomas nash a new york presbyterian hospital internist pulmonologist and infectious disease specialist told reuters the breath test was just made updifferent posts mistakenly source the claim to an unnamed stanford hospital board member a japanese doctor or taiwanese experts on march 13 stanford university tweeted misinformation about covid19 symptoms and treatment falsely attributed to stanford is circulating on social media and in email forwards it is not from stanfordstanford health care spokeswoman lisa kim told cnn on march 17 that the dangerous claim was not from stanford medicine and contains inaccurate informationthe 

In [43]:
porter = PorterStemmer()
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

In [44]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                        tokenizer=tokenizer_porter,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)
X = tfidf.fit_transform(df['title_text'])
y = df.label.values

In [45]:
X.shape

(1164, 27699)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3, shuffle=False)

In [53]:
clf = LogisticRegressionCV(cv=5, scoring='accuracy', random_state=0, n_jobs=-1,verbose=0, max_iter=300)
clf.fit(X_train, y_train)
fake_news_model = open('fake_news_model.sav', 'wb')
pickle.dump(clf, fake_news_model)
fake_news_model.close()

In [54]:
filename = 'fake_news_model.sav'
saved_clf = pickle.load

In [55]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = clf.predict(X_test)
print("---Test Set Result---")
print(classification_report(y_test, y_pred))

---Test Set Result---
              precision    recall  f1-score   support

        FAKE       0.62      0.74      0.68       103
        TRUE       0.94      0.96      0.95       174
        fake       0.54      0.37      0.44        73

    accuracy                           0.77       350
   macro avg       0.70      0.69      0.69       350
weighted avg       0.76      0.77      0.76       350

